# Comparison CellRank on RNA vs. MultiVelo + CellRank on RNA+ATAC
## Preprocessing

We will use the embryonic E18 mouse brain from 10X Multiome as an example.

CellRanger output files can be downloaded from [10X website](https://www.10xgenomics.com/resources/datasets/fresh-embryonic-e-18-mouse-brain-5-k-1-standard-1-0-0). Crucially, the filtered feature barcode matrix folder, ATAC peak annotations TSV, and the feature linkage BEDPE file in the secondary analysis outputs folder will be needed in this demo. 

Quantified unspliced and spliced counts from Velocyto can be downloaded from [MultiVelo GitHub page](https://github.com/welch-lab/MultiVelo).

We provide the cell annotations for this dataset in "cell_annotations.tsv" on the GitHub page. (To download from GitHub, click on the file, then click "Raw" on the top right corner. If it opens in your browser, you can download the page as a text file.)

Weighted nearest neighbors from Seurat can be downloaded from GitHub folder "seurat_wnn", which contains a zip file of three files: "nn_cells.txt", "nn_dist.txt", and "nn_idx.txt". Please unzip the archive after downloading. The R script used to generate such files can also be found in the same folder.
```
.
|-- MultiVelo_Demo.ipynb
|-- cell_annotations.tsv
|-- outs
|   |-- analysis
|   |   `-- feature_linkage
|   |       `-- feature_linkage.bedpe
|   |-- filtered_feature_bc_matrix
|   |   |-- barcodes.tsv.gz
|   |   |-- features.tsv.gz
|   |   `-- matrix.mtx.gz
|   `-- peak_annotation.tsv
|-- seurat_wnn
|   |-- nn_cells.txt
|   |-- nn_dist.txt
|   `-- nn_idx.txt
`-- velocyto
    `-- 10X_multiome_mouse_brain.loom
```

In [3]:
import os
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import scanpy as sc
import scvelo as scv
import cellrank as cr
import multivelo as mv

In [2]:
scv.settings.verbosity = 3
scv.settings.presenter_view = True
scv.set_figure_params('scvelo')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
np.set_printoptions(suppress=True)

In [4]:
data_path = '../data'
result_path = os.path.join(data_path, 'results')

## Preprocessing

In [11]:
def load_adata_rna():
    ## Reading in unspliced and spliced counts
    rna_fn = os.path.join(data_path, "velocyto/10X_multiome_mouse_brain.loom")
    adata_rna = scv.read(rna_fn, cache=True)
    adata_rna.obs_names = [x.split(':')[1][:-1] + '-1' for x in adata_rna.obs_names]
    adata_rna.var_names_make_unique()
    
    ## Filtering
    sc.pp.filter_cells(adata_rna, min_counts=1000)
    sc.pp.filter_cells(adata_rna, max_counts=20000)

    # Top 1000 variable genes are used for downstream analyses.
    scv.pp.filter_and_normalize(adata_rna, min_shared_counts=10, n_top_genes=1000)

    # Load cell annotations
    cell_annot_fn = os.path.join(data_path, 'cell_annotations.tsv')
    cell_annot = pd.read_csv(cell_annot_fn, sep='\t', index_col=0)

    adata_rna = adata_rna[cell_annot.index,:]
    adata_rna.obs['celltype'] = cell_annot['celltype']

    # We subset for lineages towards neurons and astrocytes.
    adata_rna = adata_rna[adata_rna.obs['celltype'].isin(['RG, Astro, OPC', 
                                                          'IPC', 
                                                          'V-SVZ', 
                                                          'Upper Layer', 
                                                          'Deeper Layer', 
                                                          'Ependymal cells', 
                                                          'Subplate'])]
    return adata_rna

In [12]:
def load_adata_atac():
    atac_fn = os.path.join(data_path, 'outs/filtered_feature_bc_matrix/')
    adata_atac = sc.read_10x_mtx(atac_fn, var_names='gene_symbols', cache=True, gex_only=False)
    adata_atac = adata_atac[:,adata_atac.var['feature_types'] == "Peaks"]

    # We aggregate peaks around each gene as well as those that have high correlations with 
    # promoter peak or gene expression.
    # Peak annotation contains the metadata for all peaks.
    # Feature linkage contains pairs of correlated genomic features.
    peak_fn = os.path.join(data_path, 'outs/peak_annotation.tsv')
    linkage_fn = os.path.join(data_path, 'outs/analysis/feature_linkage/feature_linkage.bedpe')
    adata_atac = mv.aggregate_peaks_10x(adata_atac, peak_fn, linkage_fn, verbose=True)

    sc.pp.filter_cells(adata_atac, min_counts=2000)
    sc.pp.filter_cells(adata_atac, max_counts=60000)

    # We normalize aggregated peaks with TF-IDF.
    mv.tfidf_norm(adata_atac)
    
    return adata_atac

In [13]:
def clean_rna_and_atac(adata_rna, adata_atac):
    shared_cells = pd.Index(np.intersect1d(adata_rna.obs_names, adata_atac.obs_names))
    shared_genes = pd.Index(np.intersect1d(adata_rna.var_names, adata_atac.var_names))

    # We reload in the raw data and continue with a subset of cells.
    
    rna_fn = os.path.join(data_path, "velocyto/10X_multiome_mouse_brain.loom")
    adata_rna = scv.read(rna_fn, cache=True)
    adata_rna.obs_names = [x.split(':')[1][:-1] + '-1' for x in adata_rna.obs_names]
    adata_rna.var_names_make_unique()

    adata_rna = adata_rna[shared_cells, shared_genes]
    adata_atac = adata_atac[shared_cells, shared_genes]

    scv.pp.normalize_per_cell(adata_rna)
    scv.pp.log1p(adata_rna)
    scv.pp.moments(adata_rna, n_pcs=30, n_neighbors=50)
    
    cell_annot_fn = os.path.join(data_path, 'cell_annotations.tsv')
    cell_annot = pd.read_csv(cell_annot_fn, sep='\t', index_col=0)
    
    adata_rna.obs['celltype'] = cell_annot.loc[adata_rna.obs_names, 'celltype']
    adata_rna.obs['celltype'] = adata_rna.obs['celltype'].astype('category')

    # Reorder the categories for color consistency with the manuscript.
    all_clusters = ['Upper Layer',
                    'Deeper Layer',
                    'V-SVZ',
                    'RG, Astro, OPC',
                    'Ependymal cells',
                    'IPC',
                    'Subplate']
    adata_rna.obs['celltype'] = adata_rna.obs['celltype'].cat.reorder_categories(all_clusters)

    scv.tl.umap(adata_rna)

    # Write out filtered cells and prepare to run Seurat WNN --> R script can be found on Github.
    adata_rna.obs_names.to_frame().to_csv(os.path.join(data_path, 'filtered_cells.txt'),
                                          header=False, index=False)

    # Read in Seurat WNN neighbors.
    seurat_path = os.path.join(data_path, 'seurat_wnn')
    nn_idx = np.loadtxt(os.path.join(seurat_path, "nn_idx.txt"), delimiter=',')
    nn_dist = np.loadtxt(os.path.join(seurat_path, "nn_dist.txt"), delimiter=',')
    nn_cells = pd.Index(pd.read_csv(os.path.join(seurat_path, "nn_cells.txt"), header=None)[0])

    # Make sure cell names match.
    assert np.all(nn_cells == adata_atac.obs_names)

    mv.knn_smooth_chrom(adata_atac, nn_idx, nn_dist)
    
    return adata_rna, adata_atac

In [14]:
def load_rna_and_atac():
    adata_rna = load_adata_rna()
    adata_atac = load_adata_atac()
    
    adata_rna, adata_atac = clean_rna_and_atac(adata_rna, adata_atac)
    
    return adata_rna, adata_atac

In [17]:
%%time
adata_rna, adata_atac = load_rna_and_atac()

In [22]:
adata_rna.write(os.path.join(result_path, "adata_rna_mouse_brain.h5ad"))
adata_atac.write(os.path.join(result_path, "adata_atac_mouse_brain.h5ad"))